### Instalación de librerías básicas

In [ ]:
!pip install transformers argilla datasets

In [ ]:
!pip uninstall -y huggingface_hub && pip3 install git+https://github.com/huggingface/huggingface_hub

In [ ]:
!pip install -q bitsandbytes accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

### Fine-tuning a Bertin usando LoRA

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import requests
from transformers import BertTokenizerFast, EncoderDecoderModel, AutoTokenizer, AutoConfig, AutoModelForCausalLM
from datasets import DatasetDict, Dataset, load_dataset
import argilla as rg
import pandas as pd
import transformers
import os
import torch.nn as nn
import bitsandbytes as bnb
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
model_id = 'bertin-project/bertin-gpt-j-6B'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             load_in_8bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

new_tokens = ["<SH>", "<EH>"]
num_added_toks = tokenizer.add_tokens(new_tokens)
print(f"Whe have added {num_added_toks} tokens")
model.resize_token_embeddings(len(tokenizer))

Whe have added 2 tokens


Embedding(50259, 4096)

### Postprocesado del modelo

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


### Aplicar LoRA


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7340032 || all params: 6057067603 || trainable%: 0.12118127914511903


### Preprocesado del dataset

In [ ]:
dataset = load_dataset('hackathon-somos-nlp-2023/informes_discriminacion_gitana')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating valid split:   0%|          | 0/99 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1791 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/qblocks/.cache/huggingface/datasets/hackathon-somos-nlp-2023___parquet/hackathon-somos-nlp-2023--informes_discriminacion_gitana-bd732aeefd500087/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 1791
    })
    test: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 99
    })
})

In [ ]:
vacio = {"hechos": [],
        "intervencion": [],
        "resultado": []}
for i, data in enumerate(dataset["train"]):
    if len(data["text"]) == 0:
        vacio["hechos"].append(i)
    if len(data["intervencion"]) == 0:
        vacio["intervencion"].append(i)
    if len(data["resultado"]) == 0:
        vacio["resultado"].append(i)

In [ ]:
vacio

{'hechos': [],
 'intervencion': [119,
  209,
  311,
  565,
  851,
  869,
  1091,
  1187,
  1354,
  1403,
  1460,
  1491,
  1520,
  1523,
  1625,
  1703],
 'resultado': []}

Eliminamos las intervenciones vacías

In [ ]:
dataset["train"] = dataset["train"].select(
    (
        i for i in range(len(dataset["train"])) if i not in set(vacio["intervencion"])
    )
)

[04/06/23 12:27:47] WARNING  WARNING:datasets.fingerprint:Parameter 'indices'=<generator object  ]8;id=734194;file:///home/qblocks/.local/lib/python3.8/site-packages/datasets/fingerprint.py\fingerprint.py]8;;\:]8;id=824363;file:///home/qblocks/.local/lib/python3.8/site-packages/datasets/fingerprint.py#327\327]8;;\
                             <genexpr> at 0x7f20d4f02820> of the transform                                         
                             datasets.arrow_dataset.Dataset.select couldn't be hashed properly,                    
                             a random hash was used instead. Make sure your transforms and                         
                             parameters are serializable with pickle or dill for the dataset                       
                             fingerprinting and caching to work. If you reuse this transform,                      
                             the caching mechanism will consider it to be different from the                       
                             previous calls and recompute everything. This warning is only                         
                             showed once. Subsequent hashing failures won't be showed.                             

In [ ]:
dataset["train"]

Dataset({
    features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
    num_rows: 1775
})

In [ ]:
datos_positivo = dataset.filter(lambda x: x["resultado"] == "Positivo.")
datos_negativo = dataset.filter(lambda x: x["resultado"] == "Negativo.")
datos_neutro = dataset.filter(lambda x: x["resultado"] == "Neutro.")

Filter:   0%|          | 0/1775 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1775 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1775 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
datos_positivo

DatasetDict({
    train: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 246
    })
    test: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 11
    })
    valid: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 11
    })
})

In [ ]:
# dividir el subconjunto de no positivo en dos subconjuntos aleatorios del mismo tamaño
datos_neg_train = datos_negativo["train"].train_test_split(train_size=len(datos_positivo["train"]))
datos_neu_train = datos_neutro["train"].train_test_split(train_size=len(datos_positivo["train"]))

datos_neg_test = datos_negativo["test"].train_test_split(train_size=len(datos_positivo["test"]))
datos_neu_test = datos_neutro["test"].train_test_split(train_size=len(datos_positivo["test"]))

datos_neg_valid = datos_negativo["valid"].train_test_split(train_size=len(datos_positivo["valid"]))
datos_neu_valid = datos_neutro["valid"].train_test_split(train_size=len(datos_positivo["valid"]))

In [ ]:
datos_neg_train["train"], datos_neg_test["train"], datos_neg_valid["train"]
datos_neu_train["train"], datos_neu_test["train"], datos_neu_valid["train"]

(Dataset({
     features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
     num_rows: 246
 }),
 Dataset({
     features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
     num_rows: 11
 }),
 Dataset({
     features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
     num_rows: 11
 }))

Como el dataset no está balanceado, vamos a tomar el mismo número de muestras positivas, negativas y neutras.

In [ ]:
from datasets import concatenate_datasets

datos_neg = DatasetDict({"train": datos_neg_train["train"], "test": datos_neg_test["train"], "valid": datos_neg_valid["train"]})
datos_neu = DatasetDict({"train": datos_neu_train["train"], "test": datos_neu_test["train"], "valid": datos_neu_valid["train"]})
concatenate_datasets([datos_positivo["train"], datos_neu["train"], datos_neg["train"]])

In [ ]:
balanced_dataset = DatasetDict({
                               "train": concatenate_datasets([datos_positivo["train"], datos_neu["train"], datos_neg["train"]]),
                               "test": concatenate_datasets([datos_positivo["test"], datos_neu["test"], datos_neg["test"]]),
                               "valid": concatenate_datasets([datos_positivo["valid"], datos_neu["valid"], datos_neg["valid"]])
                               })

In [ ]:
balanced_dataset = balanced_dataset.shuffle(seed=42)

In [ ]:
balanced_dataset["train"][0]

{'sintetico': '0',
 'text': 'La Voz de Galicia publicó una noticia sobre una detención policial en un barrio de Lugo. En el cuerpo de la noticia se mencionaba que los delincuentes eran de etnia gitana.',
 'intervencion': 'Se envió carta de queja al medio.',
 'tipo_discriminacion': 'Discriminación directa',
 'resultado': 'Negativo.'}

In [ ]:
def format_ds(example):
  example["input_text"] = "<SH>" + example["text"] + " Intervención: " + example["intervencion"] + " Resultado:" + example["resultado"] + "<EH>"
  return example

In [ ]:
balanced_dataset = balanced_dataset.map(format_ds, remove_columns=["sintetico", "text", "intervencion", "tipo_discriminacion", "resultado"])

Map:   0%|          | 0/738 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
balanced_dataset["train"][0]

{'input_text': '<SH>La Voz de Galicia publicó una noticia sobre una detención policial en un barrio de Lugo. En el cuerpo de la noticia se mencionaba que los delincuentes eran de etnia gitana. Intervención: Se envió carta de queja al medio. Resultado:Negativo.<EH>'}

In [ ]:
balanced_dataset = balanced_dataset.map(lambda samples: tokenizer(samples["input_text"]), batched=True)

Map:   0%|          | 0/738 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

### Configuración de los parámetros de entrenamiento

In [ ]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=balanced_dataset['train'],
    eval_dataset=balanced_dataset['test'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=1000, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=20, 
        output_dir='outputs',
        report_to='tensorboard'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

/home/qblocks/.local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
20,1.664800
40,1.548200
60,1.319800
80,1.266100
100,1.217100
120,1.178800
140,1.189100
160,1.116100
180,1.111500
200,1.065000


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1000, training_loss=0.6538836755752564, metrics={'train_runtime': 9209.8727, 'train_samples_per_second': 1.737, 'train_steps_per_second': 0.109, 'total_flos': 2.6633005587904467e+17, 'train_loss': 0.6538836755752564, 'epoch': 21.62})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 9.20


La medida de perplexity (perplejidad en español) se utiliza en el campo del procesamiento del lenguaje natural para evaluar la calidad de un modelo de lenguaje. Básicamente, la perplexity se refiere a cuán bien un modelo de lenguaje puede predecir una secuencia de palabras desconocidas, en función de su experiencia previa.

En términos generales, cuanto más bajo sea el valor de la perplexity, mejor será el modelo de lenguaje. Un modelo con una perplexity baja tiene una mejor capacidad para predecir la probabilidad de una secuencia de palabras desconocidas. Por otro lado, un modelo con una perplexity alta indica que tiene dificultades para hacer predicciones precisas.

Nuestro modelo tiene un valor de perplexity de 9.2

### Subida del modelo al hub

In [ ]:
model.push_to_hub("rwheel/discriminacion_gitana_intervenciones_balanceado", use_auth_token=True)

tokenizer.push_to_hub("rwheel/discriminacion_gitana_intervenciones_balanceado", use_auth_token=True)

## Cargar el adapter del Hub

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "rwheel/discriminacion_gitana_intervenciones"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
model.config.use_cache = False

In [ ]:
def predecir_intervencion(text):
  text = "<SH>" + text + " Intervención: "
  batch = tokenizer(text, return_tensors='pt')
  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=256, eos_token_id=50258)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=False)

In [ ]:
from datasets import load_dataset
ldataset = load_dataset('rwheel/informes_discriminacion_gitana')

[04/06/23 15:04:38] WARNING  WARNING:datasets.builder:Found cached dataset parquet                   ]8;id=234053;file:///home/qblocks/.local/lib/python3.8/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=146316;file:///home/qblocks/.local/lib/python3.8/site-packages/datasets/builder.py#817\817]8;;\
                             (/home/qblocks/.cache/huggingface/datasets/rwheel___parquet/rwheel--inf               
                             ormes_discriminacion_gitana-7a3ddd9628f402a3/0.0.0/2a3b91fbd88a2c90d1db               
                             bb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)                                         

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ldataset["valid"][9]["text"]

'Una familia gitana que reside en Mérida decide viajar a visitar a un familiar que está enfermo en Zalamea de la Serena. Durante el trayecto, hacen una parada para preguntar por qué carretera continuar cuando de pronto son parados por dos furgonetas de la guardia civil. Nos informan de que fueron cacheados todos los miembros de la familia, incluso los jóvenes que viajaban con ellos. Cuando la guardia civil comprobó por sus propios testimonios que iban a ver a un familiar que estaba enfermo, les dejo seguir su viaje. A pesar de pedir una explicación de por qué los cachearon los guardia civiles no les comentaron nada. Nos trasladan que se sintieron muy mal, porque en cuanto la guardia civil se dio cuenta que eran gitanos, los pararon. Era de noche y por miedo, no identificaron ni los vehículos ni a los agentes.'

In [ ]:
hechos = ldataset["valid"][9]["text"]
output = predecir_intervencion(hechos)

NameError: name 'tokenizer' is not defined

### Interfaz con gradio

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "hackathon-somos-nlp-2023/discriminacion_gitana_intervenciones_balanceado"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

def predecir_intervencion(text):
    text = "<SH>" + text + " Intervención: "
    batch = tokenizer(text, return_tensors='pt')
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=256, eos_token_id=50258)

    output = tokenizer.decode(output_tokens[0], skip_special_tokens=False)

    aux = output.split("Intervención:")[1].strip()
    intervencion = aux.split("Resultado:")[0].strip()
    resultado = aux.split("Resultado:")[1].split("<EH>")[0].strip()
    
    return intervencion, resultado

with gr.Blocks() as demo:
    gr.Markdown("Predicción de intervenciones para mitigar el daño racista en el pueblo gitano")
    with gr.Row():
        hechos = gr.Textbox(placeholder="Un alumno gitano de un Instituto...", label="Hechos")
    with gr.Row():
        intervencion = gr.Textbox(label="Intervención")
        resultado = gr.Textbox(label="Resultado")
        
    btn = gr.Button("Go")
    btn.click(fn=predecir_intervencion, inputs=hechos, outputs=[intervencion, resultado])

demo.launch(share=True)

2023-04-07 09:37:22.932330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 110
CUDA SETUP: Loading binary /home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda110.so...


/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4aeb53c44685dcd858.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.
/home/qblocks/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/qblocks/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Traceback (most recent call last):
  File "/home/qblocks/.local/lib/python3.8/site-packages/gradio/routes.py", line 393, in run_predict
    outpu

In [ ]:
Una joven gitana fue a hacer la compra acompañada de su hija de 12 años y su sobrina de ocho. Mientras compraban, el vigilante de seguridad no dejó de seguirlas en todo momento.